In [ ]:
# Tried FineTuning but this code didn't work due to dependencies issues with gpu.


!pip install python-docx tiktoken

In [ ]:
!pip install --upgrade transformers
! pip install transformers accelerate peft datasets bitsandbytes


# import os
# from huggingface_hub import login
# from datasets import load_dataset
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     BitsAndBytesConfig,
#     Trainer,
#     TrainingArguments
# )
# from peft import LoraConfig, get_peft_model, TaskType

# # 1. Authenticate
# os.environ["HF_TOKEN"] = ""
# login(token=os.environ["HF_TOKEN"])

# # 2. Load dataset
# TRAIN_FILE = "/content/train.jsonl"
# dataset = load_dataset("json", data_files={"train": TRAIN_FILE})["train"]

# # 3. Tokenizer + Model w/ 4-bit quantization
# MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False, use_auth_token=True)
# tokenizer.pad_token = tokenizer.eos_token

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype="float16",
#     bnb_4bit_quant_type="nf4",
# )

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_ID,
#     device_map="auto",
#     quantization_config=bnb_config,
#     use_auth_token=True
# )

# # 4. Attach LoRA
# peft_config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM,
#     inference_mode=False,
#     r=8, lora_alpha=32, lora_dropout=0.1,
# )
# model = get_peft_model(model, peft_config)

# # 5. Tokenization helper for fine-tuning
# def tokenize_batch(ex):
#     txts = [
#         p + c + tokenizer.eos_token
#         for p, c in zip(ex["prompt"], ex["completion"])
#     ]
#     batch = tokenizer(
#         txts, truncation=True, max_length=512, padding="max_length"
#     )
#     batch["labels"] = batch["input_ids"].copy()
#     return batch

# tokenized = dataset.map(
#     tokenize_batch,
#     batched=True,
#     remove_columns=["prompt", "completion"]
# )

# # 6. Trainer setup
# training_args = TrainingArguments(
#     output_dir="tiny_llama_finetuned",
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=8,
#     num_train_epochs=3,
#     learning_rate=2e-4,
#     fp16=True,
#     optim="paged_adamw_32bit",
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized,
#     tokenizer=tokenizer,
# )

# # 7. Train!
# trainer.train()

# # 8. Save adapter + tokenizer
# model.save_pretrained("tiny_llama_finetuned/lora_adapter")
# tokenizer.save_pretrained("tiny_llama_finetuned/tokenizer")


In [ ]:
================================================ERROR=====================================
CUDA SETUP: CUDA detection failed! Possible reasons:
1. CUDA driver not installed
2. CUDA not installed
3. You have multiple conflicting CUDA libraries
4. Required library not pre-compiled for this bitsandbytes release!
CUDA SETUP: If you compiled from source, try again with `make CUDA_VERSION=DETECTED_CUDA_VERSION` for example, `make CUDA_VERSION=113`.
CUDA SETUP: The CUDA version for the compile might depend on your conda install. Inspect CUDA version via `conda list | grep cuda`.
================================================================================

In [ ]:
# Understanding LoRA Adapter & Tokenizer

'''
1. tokenizer/
This directory holds all the files your model needs to turn text into tokens (and back). Typical contents include:

tokenizer.json
A consolidated JSON with all special tokens, merges, and vocabulary data.

vocab.json, merges.txt, or vocab.txt
Depending on the tokenizer type (BPE, WordPiece, etc.), these map strings ↔ token IDs.

tokenizer_config.json
Configuration flags (lowercasing, special tokens, truncation/padding defaults).

special_tokens_map.json
A map of which token IDs correspond to <pad>, <unk>, <eos>, etc.

When you later do:

tokenizer = AutoTokenizer.from_pretrained("tiny_llama_cpu_finetuned/tokenizer")
the AutoTokenizer reads exactly these files and reconstructs the same tokenizer you used for training.



2. lora_adapter/
This directory holds only the LoRA adapter weights that you trained. LoRA (Low-Rank Adapters) injects small additional weight matrices into the base model, so you don’t have to save or re-train the full 1.1 B parameters.

Typical contents include:

adapter_model.bin or pytorch_model.bin
The learned LoRA weights (usually only a few megabytes).

adapter_config.json
Tells the PEFT library how you configured LoRA (rank = 8, alpha = 32, dropout = 0.1, which layers to apply, etc.).

config.json
A tiny snippet describing how PEFT should merge these adapters with the base model at inference time.

When you load for inference:

base = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = PeftModel.from_pretrained(base, "tiny_llama_cpu_finetuned/lora_adapter")
'''